# 1. Setup Neuroglancer

In [1]:
import neuroglancer
import h5py
import numpy as np
import sys

ip='140.247.107.75'
pp=9991
neuroglancer.set_server_bind_address(bind_address=ip,bind_port=pp)
viewer=neuroglancer.Viewer()
res=[8,8,30]
mag=[200,200,14]#margin

In [9]:
# data: zyx-order
D0 = '/mnt/coxfs01/vcg_connectomics/cerebellum/gt-syn/'
print "load image"

with h5py.File(D0+'syn_0613_im_ip150.h5') as fid:
    img = np.array(fid['main'])[mag[0]:-mag[0],mag[1]:-mag[1],mag[2]:-mag[2]]

with viewer.txn() as s:
    s.layers.append(
        name='img',
        layer=neuroglancer.LocalVolume(
            data=img.transpose((2,1,0)),
            voxel_size=res,
        ))

with h5py.File(D0+'syn_0613_seg.h5') as fid:
    seg = np.array(fid['main'])[mag[0]:-mag[0],mag[1]:-mag[1],mag[2]:-mag[2]]
with viewer.txn() as s:
    s.layers.append(
        name='seg',
        layer=neuroglancer.LocalVolume(
            data=seg.transpose((2,1,0)).astype(np.uint16),
            voxel_size=res,
        ))

In [7]:
D1='/mnt/coxfs01/donglai/cerebellum/db/syn0619/'

with h5py.File(D1+'volume_0_uint8.h5') as fid:
    pred = np.array(fid['main'])
    pred=pred.transpose((2,1,0))[mag[0]:-mag[0],mag[1]:-mag[1],mag[2]:-mag[2]]

thres = 255*0.8
with viewer.txn() as s:
    s.layers.append(
        name='pred',
        layer=neuroglancer.LocalVolume(
            data=(pred.transpose((2,1,0))>=thres).astype(np.uint16),
            voxel_size=res,
        ))

In [5]:
print viewer

http://140.247.107.75:9991/v/c07a9c1b832d651f0022fa70256171c40cb4e088/


# 2. Eval GT

In [25]:
# precision-recall by pixel
"""
p1 = (seg>0)
p2 = (pred>204)
TP = np.count_nonzero(np.logical_and(p1,p2))
FP = np.count_nonzero(np.logical_and(np.logical_not(p1),p2))
T = np.count_nonzero(p1)
"""
pre = float(TP)/(TP+FP)
rec = float(TP)/T
print pre,rec

0.187540385211 0.423263999543


In [24]:
# precision-recall by segment

1051195

In [8]:
pred.shape, seg.shape,img.shape

((1946, 1808, 745), (1946, 1808, 745), (1946, 1808, 745))

In [12]:
ls /mnt/coxfs01/vcg_connectomics/cremi/

align/  gt-seg/  gt-syn/  images/  scripts/
